In [1]:
from pandas import DataFrame
from math import ceil,e
import time
import pandas as pd
import numpy as np
import os
import copy
import random

In [2]:
def shuffle_df_cols(dataframe,clas = 'class'):
    '''dataframe with class'''
    x = list(dataframe.columns)
    if clas in x:
        x.remove(clas)
        #print(1)
    random.shuffle(x)
    x.append(clas)  #put the class attr to the last position
    return x

In [3]:
def getPosNumByEqulance_hash2(dataframe,cols=[],dec_attrs='class'):
    '''return a dict and |POSB(D)|,where B=con_attrs'''
#     print('cols: ',cols)
    clas_groups = dataframe.groupby(by=dataframe[dec_attrs]).groups;del dataframe[dec_attrs]
    if not cols:
        return dict(),0
    t = dataframe.groupby(by=cols).groups
    for key in t.keys():
        check = False
        for key2 in clas_groups:
            if set(t[key]).issubset(clas_groups[key2]):
                check = True
                break
        t[key] = (t[key],len(t[key]),check)
    Count = 0
    for key in t.keys():
        if t[key][2]:
            Count += t[key][1]
    return t,Count

In [4]:
def generalized_quick_forward_greedy_reduction(inputpath = '',dec_attrs = 'class',f=None):
    start = time.clock()
    print('start!')
    if isinstance(inputpath,str):
        dataframe = pd.read_csv(inputpath,engine='c')
    elif isinstance(inputpath,pd.core.frame.DataFrame):
        dataframe = inputpath
    reduct=set()
    attr_cols = list(dataframe.columns);
    attr_cols.remove(dec_attrs);
    attr_cols = set(attr_cols)
    clas = dataframe[dec_attrs]
    Count  = 0
    
    while 1:
        u_ = dict()  
        sigs = []
        len_new_dataframe = len(dataframe)
#         print(dataframe.index)
#         print('reduct',reduct)
        tmp = copy.deepcopy(reduct)
        t,Count = getPosNumByEqulance_hash2(dataframe,cols = list(tmp),dec_attrs = dec_attrs)
        dataframe[dec_attrs] = clas
        DF_last = Count / len_new_dataframe
        for attr in attr_cols - reduct:
            tmp = copy.deepcopy(reduct)
            tmp.add(attr)
            t,Count = getPosNumByEqulance_hash2(dataframe,cols = list(tmp),dec_attrs = dec_attrs)
#             print('tmp',tmp)
            dataframe[dec_attrs] = clas
            sig = Count / len_new_dataframe - DF_last
#             print(attr,': ','Count: ',Count,'len: ',len_new_dataframe,'DF_now: ', Count / len_new_dataframe ,DF_last)
            sigs.append((attr,sig)) 
        
            for key in t.keys():
                if t[key][2] == True:
                    u_[attr] = t[key][0]
#         print('sigs:',sigs)
        ak = sigs.index(max(sigs,key = lambda x:x[1]))
#         print(sigs)
#         print(sigs[ak][0],u_[sigs[ak][0]])
        if sigs[ak][1] > 0:
            dataframe = dataframe.drop(u_[sigs[ak][0]])
            reduct.add(sigs[ak][0])
            print(sigs[ak][0])
        elif sigs[ak][1] == 0 and len(reduct) == 0:
            reduct.add(sigs[ak][0])
#             print('elif break:',sigs)
#             print('elif final reduct',reduct)
            break
        else:
#             print('else break:',sigs)
#             print('else final reduct',reduct)
            break
#         print(len(dataframe),sigs,u_)
    #print 'forward_ spend time:%fs'%(time.clock()-start),'reduct result:',list(reduct)
    print('quick_forward spend time:{}s '.format(time.clock()-start)+'reduct result: {}\n'.format(list(reduct)))
    # print(time.clock()-start)
    #return reduct
    

In [5]:
if __name__ == "__main__":
    dataframe = pd.read_csv('tap4funboolean.csv',engine='c',dtype='str')
    cols_seq = []
    tmp = shuffle_df_cols(dataframe)
    d = dataframe[tmp]
    generalized_quick_forward_greedy_reduction(d,'class')

NameError: name 'f' is not defined